<a href="https://colab.research.google.com/github/SantinoGarofalo/CREDIT-RISK-CLASSIFICATION/blob/main/credit_risk_ml_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Sviluppo Pipeline di Machine Learning : GERMAN CREDIT DATASET**

# Credit Scoring su German Credit Dataset  
### Valutazione del rischio di credito: Good vs Bad Payers

In questo notebook sviluppiamo una **pipeline end-to-end di Machine Learning** per supportare il processo di **credit scoring**: dato il profilo di un cliente, stimiamo la probabilità che sia un:

- **Good Payer** → Basso rischio di insolvenza  
- **Bad Payer** → Alto rischio di insolvenza  

L’obiettivo non è solo raggiungere buone performance predittive, ma anche:

- garantire una **valutazione robusta e affidabile** (train/valid/test, CV stratificata, tuning iperparametri)
- mantenere **interpretabilità ed explainability** (Feature Importance, SHAP, LIME, PDP/ICE)
- collegare i risultati a una **logica di business** (trade-off tra precision/recall sui Bad Payers).


## 1. Executive Summary

Gli istituti finanziari devono decidere se **concedere o meno un credito** a un potenziale cliente.  
Una decisione errata può comportare:

- **False Negative (Bad Payer classificato come Good)** → perdita economica potenzialmente significativa  
- **False Positive (Good Payer classificato come Bad)** → mancato guadagno e peggioramento della customer experience  

In questo lavoro:

- costruiamo un **classificatore binario** su un dataset storico di clienti (German Credit Dataset, 1.000 osservazioni)
- stimiamo il **rischio di insolvenza** e analizziamo l’impatto delle principali variabili (es. durata del credito, storico dei pagamenti, importo)
- esploriamo il **trade-off fra recall e precision sui Bad Payers**, in funzione del profilo di rischio dell’istituto.


## 2. Struttura del Notebook

1. [Executive Summary](#1-Executive-Summary)  
2. [Struttura del Notebook](#2-Struttura-del-Notebook)  
3. [Setup Tecnico & Caricamento Dati](#3-Setup-Tecnico--Caricamento-Dati)  
4. [Data Understanding & Data Quality](#4-Data-Understanding--Data-Quality)  
5. [Data Engineering & Feature Engineering](#5-Data-Engineering--Feature-Engineering)  
6. [Modellazione & Strategia di Validazione](#6-Modellazione--Strategia-di-Validazione)  
7. [Model Selection & Hyperparameter Tuning](#7-Model-Selection--Hyperparameter-Tuning)  
8. [Valutazione Finale su Test Set](#8-Valutazione-Finale-su-Test-Set)  
9. [Explainability (XAI) & Risk Governance](#9-Explainability-XAI--Risk-Governance)  
10. [Conclusioni & Next Steps](#10-Conclusioni--Next-Steps)  


## 3. Setup Tecnico & Caricamento Dati

In questa sezione:

- importiamo le principali librerie per **data analysis** e **machine learning**
- fissiamo alcuni **parametri globali** (es. `RANDOM_STATE`)
- carichiamo il **German Credit Dataset** da `openml` e definiamo la variabile target.


In [ ]:
# Configurazione generale
RANDOM_STATE = 42
import warnings #libreria di warning
warnings.filterwarnings("ignore")

# Manipolazione dati
import numpy as np
import pandas as pd

# Visualizzazione
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use("default")
sns.set_theme()

# Dataset
from sklearn.datasets import fetch_openml

# Preprocessing
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score, cross_validate
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Modelli
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, IsolationForest
import xgboost as xgb
import lightgbm as lgb

# Metriche
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score, roc_auc_score,
    confusion_matrix, RocCurveDisplay, PrecisionRecallDisplay
)
!pip install xgboost lightgbm optuna shap lime --quiet

# Explainability
import shap
from lime.lime_tabular import LimeTabularExplainer
from sklearn.inspection import PartialDependenceDisplay

# Tuning
import optuna


In [ ]:
# Caricamento German Credit Dataset da OpenML
data = fetch_openml("credit-g", version=1, as_frame=True)
df_raw = data.frame.copy()

print(f"Shape originale del dataset: {df_raw.shape}")
display(df_raw.head())

### 3.1 Definizione della variabile target

La variabile target nel German Credit Dataset è `class`, con due possibili valori:

- `good` → cliente considerato affidabile
- `bad` → cliente considerato rischioso  

Per maggiore chiarezza business, la rinominiamo in **`Risk`** e la codifichiamo come:

- `0` → Good Payer  
- `1` → Bad Payer


In [ ]:
# Copia di lavoro del dataset
df = df_raw.rename(columns={"class": "Risk"}).copy()

# Analisi rapida della distribuzione della target
print("Distribuzione originale della variabile 'Risk':")
display(df["Risk"].value_counts())
display(df["Risk"].value_counts(normalize=True).rename("proportion"))

# Codifica binaria: 1 = Bad Payer, 0 = Good Payer
df["Risk_binary"] = (df["Risk"] == "bad").astype(int)
df[["Risk", "Risk_binary"]].head()

## 4. Data Understanding & Data Quality

Obiettivo di questa sezione:

- comprendere la **struttura del dataset** (tipologie di variabili, cardinalità, distribuzioni)
- verificare la presenza di **missing values** e **anomalie evidenti**
- analizzare il **bilanciamento della variabile target**, cruciale per problemi di credit scoring.


In [ ]:
# Tipologie di variabili
print("Informazioni sulle colonne:")
df.info()

# Missing values
missing = df.isna().sum()
missing = missing[missing > 0].sort_values(ascending=False)

if missing.empty:
    print("\nNessun missing value rilevato nel dataset.")
else:
    print("\nMissing values per colonna:")
    display(missing.to_frame(name="n_missing"))

### 4.1 Qualità dei dati

Dal check preliminare si osserva che:

-  *“non sono presenti valori mancanti, il che semplifica la fase di data cleaning”*  
- il dataset contiene sia **variabili numeriche** che **categoriche**, spesso codificate in forma stringa (es. stato del conto, scopo del credito, rating di credito precedente).

Questi aspetti guideranno le scelte successive di:

- **encoding** (One-Hot Encoding per le categoriche)
- **scaling** (StandardScaler per le numeriche).

In [ ]:
# Identificazione feature numeriche e categoriche (escludiamo la target binaria)
feature_cols = [c for c in df.columns if c not in ["Risk", "Risk_binary"]]

numeric_features = df[feature_cols].select_dtypes(include=[np.number]).columns.tolist()
categorical_features = df[feature_cols].select_dtypes(include=["object", "category"]).columns.tolist()

print("Feature numeriche:", numeric_features)
print("Feature categoriche:", categorical_features)

In [ ]:
fig, ax = plt.subplots(figsize=(4,4))
sns.countplot(x="Risk", data=df, ax=ax)
ax.set_title("Distribuzione della variabile target (Good vs Bad)")
ax.set_xlabel("Risk")
ax.set_ylabel("Count")
plt.show()

### 4.2 Bilanciamento della variabile target

La distribuzione di `Risk` è **sbilanciata**:

- la maggior parte dei clienti è classificata come **Good**
- la classe **Bad** è meno frequente, ma non estremamente rara.

Questo ha implicazioni importanti:

- metriche come **accuracy** possono risultare fuorvianti  
- saranno fondamentali **precision, recall, F1 e AUC**, con particolare attenzione alla **recall sulla classe Bad**  
  (mancare un Bad Payer significa potenziale perdita economica).

## 5. Data Engineering & Feature Engineering

In questa sezione applichiamo una serie di trasformazioni ai dati grezzi per renderli adatti alla modellazione:

1. **Gestione outlier** sulle variabili numeriche, combinando:
   - approccio **statistico** (Interquartile Range, IQR)
   - approccio **model-based** (Isolation Forest)

2. **Feature engineering mirata** per migliorare la capacità predittiva del modello.

3. Definizione chiara dello **schema delle feature** (numeriche vs categoriche) da utilizzare nel preprocessing.

### 5.1 Rilevamento e trattamento outlier

Nel contesto di **credit scoring**, gli outlier possono rappresentare:

- **casi realmente estremi** (es. importi molto elevati, durate particolari del credito)
- **anomalie / errori** di registrazione

In questo notebook adottiamo un approccio conservativo:

- applichiamo una **doppia logica di rilevamento** sugli attributi numerici:
  - IQR (Interquartile Range): individua valori molto distanti dalla massa dei dati
  - Isolation Forest: modello unsupervised che isola osservazioni "strane"

- rimuoviamo solo i record che risultano outlier **almeno secondo uno dei due metodi**, documentando l’impatto sul dataset.


In [ ]:
df_eng = df.copy() # Initialize df_eng by copying df

# Riconfermiamo le feature numeriche / categoriche sulla copia df_eng
feature_cols = [c for c in df_eng.columns if c not in ["Risk", "Risk_binary"]]

numeric_features = df_eng[feature_cols].select_dtypes(include=[np.number]).columns.tolist()
categorical_features = df_eng[feature_cols].select_dtypes(include=["object", "category"]).columns.tolist()

print("Feature numeriche:", numeric_features)
print("Feature categoriche:", categorical_features)


def iqr_outlier_mask(df, cols, k=1.5):
    """
    Restituisce una mask booleana degli outlier secondo regola IQR:
    valori al di fuori di [Q1 - k*IQR, Q3 + k*IQR].
    """
    mask = pd.Series(False, index=df.index)
    for c in cols:
        q1 = df[c].quantile(0.25)
        q3 = df[c].quantile(0.75)
        iqr = q3 - q1
        lower = q1 - k * iqr
        upper = q3 + k * iqr
        mask = mask | (df[c] < lower) | (df[c] > upper)
    return mask


# Mask outlier con IQR
iqr_mask = iqr_outlier_mask(df_eng, numeric_features)
print(f"Outlier rilevati con IQR: {iqr_mask.sum()} ({iqr_mask.mean():.2%} del dataset)")

# Mask outlier con Isolation Forest
iso = IsolationForest(
    contamination=0.05,
    random_state=RANDOM_STATE
)
iso_labels = iso.fit_predict(df_eng[numeric_features])
iso_mask = (iso_labels == -1)
print(f"Outlier rilevati con Isolation Forest: {iso_mask.sum()} ({iso_mask.mean():.2%} del dataset)")

# Mask combinata (logica OR)
combined_mask = iqr_mask | iso_mask
print(f"Outlier totali (IQR ∪ Isolation Forest): {combined_mask.sum()} ({combined_mask.mean():.2%} del dataset)")

In [ ]:
print(f"Shape iniziale: {df_eng.shape}")
df_clean = df_eng.loc[~combined_mask].reset_index(drop=True)
print(f"Shape dopo rimozione outlier: {df_clean.shape}")

print("\nDistribuzione 'Risk_binary' dopo la pulizia:")
display(df_clean["Risk_binary"].value_counts())
display(df_clean["Risk_binary"].value_counts(normalize=True).rename("proportion"))

**Osservazioni sugli outlier**

- La rimozione degli outlier comporta una riduzione del dataset del **27,20%**
- La distribuzione della variabile target `Risk_binary` rimane **sostanzialmente stabile**, segno che non stiamo eliminando in modo selettivo una delle due classi.
- In un contesto reale, questa scelta andrebbe discussa con il **risk management**:
  - alcuni outlier potrebbero rappresentare **casi business rilevanti** (es. high-net-worth individuals)
  - la logica di filtraggio potrebbe diventare una **regola di data quality** condivisa a livello aziendale.

### 5.2 Feature Engineering

Come esempio di feature engineering introduciamo una variabile binaria:

- `long_duration`: vale 1 se la **durata del credito** è superiore alla mediana del campione.

L’idea è catturare il fatto che **crediti più lunghi** possono essere associati a un profilo di rischio diverso, per via della maggiore incertezza sul futuro del cliente.


In [ ]:
# Creazione feature binaria sulla durata del credito
duration_median = df_clean["duration"].median()
df_clean["long_duration"] = (df_clean["duration"] > duration_median).astype(int)

# Aggiorno la lista delle feature (escludo target)
feature_cols_clean = [c for c in df_clean.columns if c not in ["Risk", "Risk_binary"]]

numeric_features = df_clean[feature_cols_clean].select_dtypes(include=[np.number]).columns.tolist()
categorical_features = df_clean[feature_cols_clean].select_dtypes(include=["object", "category"]).columns.tolist()

print("Feature numeriche (post-feature engineering):")
print(numeric_features)
print("\nFeature categoriche (post-feature engineering):")
print(categorical_features)

## 6. Modellazione & Strategia di Validazione

Per costruire e valutare i modelli seguiamo questa strategia:

- **Train (60%)**: usato per stimare i parametri dei modelli  
- **Validation (20%)**: usato per confrontare modelli / soglie / configurazioni  
- **Test (20%)**: usato solo alla fine per la valutazione definitiva

Tutti gli split sono **stratificati sulla variabile `Risk_binary`** per preservare il rapporto Good / Bad.


In [ ]:
# Matrice delle feature e target
X = df_clean[feature_cols_clean].copy()
y = df_clean["Risk_binary"].copy()

# Train+Validation vs Test (80/20)
X_train_val, X_test, y_train_val, y_test = train_test_split(
    X, y,
    test_size=0.20,
    stratify=y,
    random_state=RANDOM_STATE
)

# Train vs Validation (60/20 globale)
X_train, X_val, y_train, y_val = train_test_split(
    X_train_val, y_train_val,
    test_size=0.25,
    stratify=y_train_val,
    random_state=RANDOM_STATE
)

print("Shape Train     :", X_train.shape)
print("Shape Validation:", X_val.shape)
print("Shape Test      :", X_test.shape)

print("\nDistribuzione target per split:")
for name, target in [("Train", y_train), ("Validation", y_val), ("Test", y_test)]:
    print(f"\n{name}")
    display(target.value_counts(normalize=True).rename("proportion"))


### 6.1 Preprocessing: encoding & scaling

Definiamo un pipeline di preprocessing riutilizzabile in tutti i modelli:

- **numeriche** → `StandardScaler`
- **categoriche** → `OneHotEncoder(handle_unknown="ignore")`

Il tutto viene assemblato con `ColumnTransformer` e integrato nei modelli tramite `Pipeline`.

In [ ]:
numeric_transformer = Pipeline(steps=[
    ("scaler", StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ("onehot", OneHotEncoder(handle_unknown="ignore"))
])

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features)
    ]
)

preprocessor

### 6.2 Funzioni helper per metriche e cross-validation

Definiamo ora alcune funzioni di utilità per:

- stampare le metriche principali su un singolo set (Validation / Test)
- eseguire una **cross-validation stratificata** con più metriche contemporaneamente.

In [ ]:
def print_classification_metrics(y_true, y_pred, y_proba, prefix=""):
    if prefix:
        print(f"=== {prefix} ===")
    print(f"Accuracy : {accuracy_score(y_true, y_pred):.3f}")
    print(f"Precision: {precision_score(y_true, y_pred):.3f}")
    print(f"Recall   : {recall_score(y_true, y_pred):.3f}")
    print(f"F1-score : {f1_score(y_true, y_pred):.3f}")
    print(f"AUC      : {roc_auc_score(y_true, y_proba):.3f}")


scoring = {
    "accuracy": "accuracy",
    "precision": "precision",
    "recall": "recall",
    "f1": "f1",
    "roc_auc": "roc_auc"
}

def evaluate_model_cv(model, X, y, cv=5, scoring=scoring, random_state=RANDOM_STATE):
    skf = StratifiedKFold(
        n_splits=cv,
        shuffle=True,
        random_state=random_state
    )
    cv_results = cross_validate(
        model,
        X,
        y,
        cv=skf,
        scoring=scoring,
        n_jobs=-1
    )

    summary = {
        metric: {
            "mean": cv_results[f"test_{metric}"].mean(),
            "std": cv_results[f"test_{metric}"].std()
        }
        for metric in scoring.keys()
    }
    return summary


## 7. Model Selection & Hyperparameter Tuning

### 7.1 Modello baseline: Logistic Regression

Come baseline utilizziamo una **Logistic Regression**:

- è un modello lineare, semplice da interpretare
- è ampiamente utilizzato in ambito **credit risk** (anche per motivi regolamentari)

Tutti i modelli successivi dovranno portare un miglioramento rispetto a questa baseline, in particolare su:

- **AUC**
- **Recall sulla classe Bad (1)**.

In [ ]:
log_reg_clf = Pipeline(steps=[
    ("preprocess", preprocessor),
    ("model", LogisticRegression(
        max_iter=1000,
        class_weight="balanced",   # per dare più peso alla classe minoritaria (Bad)
        random_state=RANDOM_STATE
    ))
])

# Fit su Train
log_reg_clf.fit(X_train, y_train)

# Valutazione su Validation
y_val_pred = log_reg_clf.predict(X_val)
y_val_proba = log_reg_clf.predict_proba(X_val)[:, 1]

print_classification_metrics(y_val, y_val_pred, y_val_proba, prefix="Logistic Regression - Validation")

In [ ]:
models = {}

# Logistic Regression (baseline) anche in CV
models["LogisticRegression"] = Pipeline(steps=[
    ("preprocess", preprocessor),
    ("model", LogisticRegression(
        max_iter=1000,
        class_weight="balanced",
        random_state=RANDOM_STATE
    ))
])

# Random Forest
models["RandomForest"] = Pipeline(steps=[
    ("preprocess", preprocessor),
    ("model", RandomForestClassifier(
        n_estimators=300,
        max_depth=None,
        class_weight="balanced",
        random_state=RANDOM_STATE,
        n_jobs=-1
    ))
])

# Gradient Boosting
models["GradientBoosting"] = Pipeline(steps=[
    ("preprocess", preprocessor),
    ("model", GradientBoostingClassifier(
        random_state=RANDOM_STATE
    ))
])

# XGBoost
models["XGBoost"] = Pipeline(steps=[
    ("preprocess", preprocessor),
    ("model", xgb.XGBClassifier(
        objective="binary:logistic",
        eval_metric="logloss",
        n_estimators=300,
        learning_rate=0.05,
        max_depth=4,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=RANDOM_STATE,
        n_jobs=-1
    ))
])

# LightGBM
models["LightGBM"] = Pipeline(steps=[
    ("preprocess", preprocessor),
    ("model", lgb.LGBMClassifier(
        objective="binary",
        n_estimators=300,
        learning_rate=0.05,
        num_leaves=31,
        random_state=RANDOM_STATE,
        n_jobs=-1
    ))
])

In [ ]:
cv_results = {}

for name, model in models.items():
    print(f"Valutazione modello: {name}")
    summary = evaluate_model_cv(model, X_train_val, y_train_val, cv=5)
    cv_results[name] = summary

# Costruisco un DataFrame riassuntivo “da portfolio”
rows = []
for name, metrics_dict in cv_results.items():
    row = {"model": name}
    for metric, values in metrics_dict.items():
        row[f"{metric}_mean"] = values["mean"]
        row[f"{metric}_std"] = values["std"]
    rows.append(row)

results_df = pd.DataFrame(rows)
results_df = results_df.sort_values("roc_auc_mean", ascending=False).reset_index(drop=True)
results_df.sort_values

### 7.3 Confronto modelli

La tabella sopra riassume le performance medie in **cross-validation stratificata** (k=5) su Train+Validation.

- i modelli **tree-based** (es. Random Forest, XGBoost) hanno una **AUC superiore** rispetto alla Logistic Regression
- nel trade-off tra complessità e performance:
  - Logistic Regression rimane un **benchmark interpretabile**

Nel passo successivo:
- selezioneremo uno dei modelli migliori
- applicheremo un **tuning iperparametri con Optuna**
- valuteremo il modello finale sul **Test Set**.

### 7.4 Hyperparameter Tuning con Optuna (XGBoost)

Selezioniamo XGBoost come modello candidato principale e utilizziamo **Optuna** per:

- esplorare automaticamente lo spazio degli iperparametri
- massimizzare la metrica **ROC AUC** in cross-validation stratificata

Il tuning viene effettuato sul blocco **Train+Validation** (`X_train_val`, `y_train_val`), mantenendo il **Test Set** completamente separato.


In [ ]:
def objective(trial):
    # spazio di ricerca iperparametri XGBoost
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 200, 600),
        "max_depth": trial.suggest_int("max_depth", 2, 8),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
        "min_child_weight": trial.suggest_float("min_child_weight", 1.0, 10.0),
        "gamma": trial.suggest_float("gamma", 0.0, 5.0),
        "reg_lambda": trial.suggest_float("reg_lambda", 1e-3, 10.0, log=True),
        "reg_alpha": trial.suggest_float("reg_alpha", 1e-3, 10.0, log=True),
    }

    model = Pipeline(steps=[
        ("preprocess", preprocessor),
        ("model", xgb.XGBClassifier(
            objective="binary:logistic",
            eval_metric="logloss",
            random_state=RANDOM_STATE,
            n_jobs=-1,
            **params
        ))
    ])

    skf = StratifiedKFold(
        n_splits=3,
        shuffle=True,
        random_state=RANDOM_STATE
    )

    # cross-validation su ROC AUC
    scores = cross_val_score(
        model,
        X_train_val,
        y_train_val,
        cv=skf,
        scoring="roc_auc",
        n_jobs=-1
    )

    return scores.mean()

In [ ]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=150, show_progress_bar=True)

print("Best ROC AUC:", study.best_value)
print("Best params:")
study.best_params

### 7.5 Modello finale XGBoost

Utilizziamo ora i **migliori iperparametri** trovati da Optuna per:

1. ricostruire la pipeline `preprocess + XGBoost`
2. addestrare il modello finale su **Train+Validation** (80% dei dati)
3. mantenere il **Test Set** come stima out-of-sample.


In [ ]:
best_xgb = Pipeline(steps=[
    ("preprocess", preprocessor),
    ("model", xgb.XGBClassifier(
        objective="binary:logistic",
        eval_metric="logloss",
        random_state=RANDOM_STATE,
        n_jobs=-1,
        **study.best_params
    ))
])

best_xgb.fit(X_train_val, y_train_val)


## 8. Valutazione Finale su Test Set

Valutiamo ora il modello finale su dati mai visti (`X_test`, `y_test`):

- metriche di classificazione standard (Accuracy, Precision, Recall, F1, AUC)
- **Confusion Matrix**
- curve **ROC** e **Precision-Recall**
- analisi di **tuning della soglia di decisione** per ottimizzare il trade-off tra:
  - individuare il maggior numero possibile di **Bad Payers** (alta Recall)
  - contenere il numero di **falsi allarmi** (alta Precision).


In [ ]:
# Predizioni su Test
y_test_proba = best_xgb.predict_proba(X_test)[:, 1]
y_test_pred = (y_test_proba >= 0.5).astype(int)  # soglia standard 0.5

print_classification_metrics(y_test, y_test_pred, y_test_proba, prefix="XGBoost Finale - Test (threshold=0.5)")

In [ ]:
cm = confusion_matrix(y_test, y_test_pred, labels=[0,1])
tn, fp, fn, tp = cm.ravel()

plt.figure(figsize=(4,4))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues",
            xticklabels=["Pred Good (0)", "Pred Bad (1)"],
            yticklabels=["True Good (0)", "True Bad (1)"])
plt.show()

print("TN:", tn, "FP:", fp, "FN:", fn, "TP:", tp)


In [ ]:
# ROC Curve
RocCurveDisplay.from_predictions(y_test, y_test_proba)
plt.title("ROC Curve - Test")
plt.show()

# Precision-Recall Curve
PrecisionRecallDisplay.from_predictions(y_test, y_test_proba)
plt.title("Precision-Recall Curve - Test")
plt.show()

### 8.1 Threshold Tuning

La soglia standard di 0.5 non è necessariamente ottimale in un problema di **credit risk**.

Tipicamente:

- le banche preferiscono **non perdere Bad Payers** (alta Recall sulla classe 1)
- sono disposte a tollerare qualche **falso positivo** in più (clienti buoni classificati come rischiosi)

Analizziamo quindi le metriche al variare della soglia di decisione.

In [ ]:
thresholds = np.linspace(0.1, 0.9, 17)  # da 0.1 a 0.9 step 0.05
records = []

for thr in thresholds:
    y_thr = (y_test_proba >= thr).astype(int)
    records.append({
        "threshold": thr,
        "accuracy": accuracy_score(y_test, y_thr),
        "precision": precision_score(y_test, y_thr, zero_division=0),
        "recall": recall_score(y_test, y_thr),
        "f1": f1_score(y_test, y_thr),
    })

thr_df = pd.DataFrame(records)
thr_df


In [ ]:
plt.figure(figsize=(8,5))
plt.plot(thr_df["threshold"], thr_df["precision"], marker="o", label="Precision")
plt.plot(thr_df["threshold"], thr_df["recall"], marker="o", label="Recall")
plt.plot(thr_df["threshold"], thr_df["f1"], marker="o", label="F1-score")
plt.xlabel("Threshold")
plt.ylabel("Score")
plt.title("Metriche al variare della soglia di decisione")
plt.legend()
plt.grid(True)
plt.show()


## 9. Explainability (XAI) & Risk Governance

I modelli di credit scoring devono essere **interpretabili**:

- per ragioni normative (Basilea / EBA Guidelines)
- per supportare i **credit officer**
- per garantire trasparenza verso i clienti (“adverse action notice”)

In questa sezione forniamo spiegazioni **globali** e **locali** del modello finale XGBoost.

In [ ]:
# Estrazione del modello "puro" dal pipeline
xgb_model = best_xgb.named_steps["model"]

# Bisogna anche estrarre i nomi delle feature dopo one-hot encoding
preprocessor_fitted = best_xgb.named_steps["preprocess"]

# Recuperiamo nomi numerici
num_feats = numeric_features.copy()

# Recuperiamo nomi categorici (one-hot)
ohe = preprocessor_fitted.named_transformers_["cat"].named_steps["onehot"]
cat_feats = ohe.get_feature_names_out(categorical_features).tolist()

# Feature names finale
feature_names = num_feats + cat_feats

# Importance
importances = xgb_model.feature_importances_

fi_df = pd.DataFrame({
    "feature": feature_names,
    "importance": importances
}).sort_values("importance", ascending=False).head(20)

plt.figure(figsize=(8,6))
sns.barplot(data=fi_df, x="importance", y="feature")
plt.title(" Feature Importances (XGBoost)")
plt.tight_layout()
plt.show()


### 9.2 SHAP Values (global & local)

SHAP permette di:

- spiegare il contributo di ciascuna feature
- sia a livello **globale** (summary plot)
- sia a livello **individuale** (waterfall plot)

Nel contesto creditizio:
- valori SHAP positivi aumentano la stima di **rischio (Bad)**
- valori SHAP negativi spingono verso **Good**.

In [ ]:
shap.initjs()

# Trasformazione del test set tramite preprocessing
X_test_trans = preprocessor_fitted.transform(X_test)

# SHAP explainer basato su XGBoost
explainer = shap.TreeExplainer(xgb_model)
shap_values = explainer.shap_values(X_test_trans)


In [ ]:
shap.summary_plot(shap_values, X_test_trans, feature_names=feature_names)


### 9.4 LIME (Local Interpretable Model-agnostic Explanations)

LIME genera una spiegazione locale simulando il comportamento del modello
attorno ad un punto specifico.

È utile come:
- strumento didattico
- spiegazione leggibile da **analisti di rischio** e stakeholder meno tecnici

In [ ]:
# Trasformazione train set
X_train_trans = preprocessor_fitted.transform(X_train)

# Creazione explainer LIME
explainer_lime = LimeTabularExplainer(
    training_data=X_train_trans.toarray() if hasattr(X_train_trans, "toarray") else X_train_trans,
    feature_names=feature_names,
    class_names=["Good", "Bad"],
    discretize_continuous=True
)

idx = 1

# Spiegazione sullo stesso individuo SHAP (idx)
x_instance_lime = X_test_trans[idx].toarray()[0] if hasattr(X_test_trans, "toarray") else X_test_trans[idx]

exp = explainer_lime.explain_instance(
    data_row=x_instance_lime,
    predict_fn=xgb_model.predict_proba,
    num_features=20
)

exp.show_in_notebook(show_table=True)

### 9.5 Partial Dependence Plots (PDP) & ICE Plots

I PDP e ICE consentono di analizzare l'effetto di **una singola feature**
sulla probabilità di default, mantenendo fisse le altre variabili.

Lettura tipica:
- un PDP crescente suggerisce che feature ↑ ⇒ rischio ↑  
- un PDP decrescente suggerisce che feature ↑ ⇒ rischio ↓  

Gli ICE mostrano come **singoli individui** si comportano rispetto alla media.

In [ ]:
# Scegli una feature numerica rilevante
target_feature = numeric_features[0]  # esempio: "duration"

fig, ax = plt.subplots(figsize=(6,4))
PartialDependenceDisplay.from_estimator(
    best_xgb,
    X_test,
    features=[target_feature],
    ax=ax
)
plt.title(f"PDP per {target_feature}")
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(6,4))
PartialDependenceDisplay.from_estimator(
    best_xgb,
    X_test,
    features=[target_feature],
    kind="individual",   # ICE
    ax=ax
)
plt.title(f"ICE Plot per {target_feature}")
plt.show()